In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
import numpy as np
import os
from warnings import filterwarnings
filterwarnings('ignore')

In [6]:
batch_size = 64 # Setting batch-size for model
epochs = 100 # Epochs to run during training
latent_dim = 250 # Vector dimensions for each character
num_samples = 20000 # samples to be trained on
path = "C:/BEPEC Python Material/DataSets , NLP datasets/"
for i in os.listdir(path):
    if 'French' in i:
        data_path = path+i
print(data_path)

C:/BEPEC Python Material/DataSets , NLP datasets/French-to-English data.txt


                    Reading Data character wise as input_text as well as target_texts

In [13]:
input_texts = []
target_texts = []

input_characters = set()
target_characters = set()

# Opening file
with open(data_path , 'r' , encoding='utf-8') as f:
    lines = f.read().split('\n')

# reading 20000 lines
for line in lines[: min(num_samples , len(lines)-1)]:
    input_text , target_text , _ = line.split('\t')
    
    # We used 'tab' and '\n' as start and end sequence characters for target texts
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    # Adding unique characters of both languages
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
    
print(input_characters)

{'J', 'b', 'K', 'W', ',', '"', ' ', 'a', '9', '2', 'T', "'", 's', '5', 'G', 'V', 'n', 'A', 'O', 'k', '4', '-', 'h', 'j', 'o', 'z', 'y', 'M', 'P', 'u', 'H', 'w', 'i', ':', '%', 'D', 'f', 'Q', '$', '.', '1', 'm', 'F', '6', 'e', 'U', 'é', 'v', 'q', 'S', 'g', 'E', 'd', 'l', 'c', 'C', 'B', 'L', 'N', 'r', '0', '&', 'Y', '3', 'R', '!', 't', 'I', 'p', '?', '8', 'x', '7'}


In [14]:
print(target_texts[:5])

['\tVa !\n', '\tMarche.\n', '\tBouge !\n', '\tSalut !\n', '\tSalut.\n']


In [15]:
print(target_characters)

{'\u2009', ',', ' ', "'", 'G', 'M', 'P', 'H', 'î', ':', '\xa0', '%', 'm', 'F', 'é', 'q', 'E', 'l', '‘', 'r', 'Y', '?', 'œ', 'a', 's', 'À', 'ç', 'n', 'O', '\u202f', '-', 'ù', 'o', 'z', 'Ç', 'i', '$', '.', 'ë', 'U', 'â', '«', 'd', 'g', 'B', 'N', '3', 'R', ')', 'p', '8', '7', 'J', 'b', 'K', '(', '2', 'T', 'V', '5', '’', 'u', 'à', 'û', 'D', 'f', 'e', '\n', 'ï', 'S', 'Ê', 'c', 'L', '&', '»', 't', 'v', 'è', 'x', 'Ô', '9', 'A', 'k', '\t', 'ê', '4', 'h', 'j', 'y', 'w', 'Q', '1', '6', 'C', '0', '!', 'I', 'É', 'ô'}


                        Calculating sequences and input/targets lenghts for input and output tensors

In [16]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_sequence_len = max([len(txt) for txt in input_texts])
max_decoder_sequence_len = max([len(txt) for txt in target_texts])

print("Number of samples ", len(input_texts))
print("Number of encoder tokens ", num_encoder_tokens)
print("Number of decoder tokens ", num_decoder_tokens)
print("Maximum input sequence lenght ", max_encoder_sequence_len)
print("Maximum output sequence lenght ", max_decoder_sequence_len)

Number of samples  20000
Number of encoder tokens  73
Number of decoder tokens  99
Maximum input sequence lenght  17
Maximum output sequence lenght  59


In [18]:
# Assigning unique integer token to each input and target Sequence

input_token_dic = dict([(char , i) for i , char in enumerate(input_characters)])
target_token_dic = dict([(char , i) for i , char in enumerate(target_characters)])
target_token_dic

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '$': 4,
 '%': 5,
 '&': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '0': 13,
 '1': 14,
 '2': 15,
 '3': 16,
 '4': 17,
 '5': 18,
 '6': 19,
 '7': 20,
 '8': 21,
 '9': 22,
 ':': 23,
 '?': 24,
 'A': 25,
 'B': 26,
 'C': 27,
 'D': 28,
 'E': 29,
 'F': 30,
 'G': 31,
 'H': 32,
 'I': 33,
 'J': 34,
 'K': 35,
 'L': 36,
 'M': 37,
 'N': 38,
 'O': 39,
 'P': 40,
 'Q': 41,
 'R': 42,
 'S': 43,
 'T': 44,
 'U': 45,
 'V': 46,
 'Y': 47,
 'a': 48,
 'b': 49,
 'c': 50,
 'd': 51,
 'e': 52,
 'f': 53,
 'g': 54,
 'h': 55,
 'i': 56,
 'j': 57,
 'k': 58,
 'l': 59,
 'm': 60,
 'n': 61,
 'o': 62,
 'p': 63,
 'q': 64,
 'r': 65,
 's': 66,
 't': 67,
 'u': 68,
 'v': 69,
 'w': 70,
 'x': 71,
 'y': 72,
 'z': 73,
 '\xa0': 74,
 '«': 75,
 '»': 76,
 'À': 77,
 'Ç': 78,
 'É': 79,
 'Ê': 80,
 'Ô': 81,
 'à': 82,
 'â': 83,
 'ç': 84,
 'è': 85,
 'é': 86,
 'ê': 87,
 'ë': 88,
 'î': 89,
 'ï': 90,
 'ô': 91,
 'ù': 92,
 'û': 93,
 'œ': 94,
 '\u2009': 95,
 '‘': 96,
 '’': 97,
 '\u202f': 98}

In [25]:
input_token_dic

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'W': 44,
 'Y': 45,
 'a': 46,
 'b': 47,
 'c': 48,
 'd': 49,
 'e': 50,
 'f': 51,
 'g': 52,
 'h': 53,
 'i': 54,
 'j': 55,
 'k': 56,
 'l': 57,
 'm': 58,
 'n': 59,
 'o': 60,
 'p': 61,
 'q': 62,
 'r': 63,
 's': 64,
 't': 65,
 'u': 66,
 'v': 67,
 'w': 68,
 'x': 69,
 'y': 70,
 'z': 71,
 'é': 72}

    Encoder_input_data must be characterwise one-hot representation of input sentences since it is character level model

In [28]:
# Creating input/output tensors to be passed 

encoder_input_data = np.zeros((len(input_texts) , max_encoder_sequence_len , num_encoder_tokens) , dtype = 'float32')
decoder_input_data = np.zeros((len(input_texts) , max_decoder_sequence_len , num_decoder_tokens) , dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts) , max_decoder_sequence_len , num_decoder_tokens) , dtype = 'float32')

print(encoder_input_data.shape,"\n")
print(decoder_input_data.shape)

(20000, 17, 73) 

(20000, 59, 99)


In [30]:
target_token_dic[' ']

2

In [31]:
# here we are actaully assigning one-hot labels for each input/output char in each input/output text .  

for i , (input_text , target_text) in enumerate(zip(input_texts , target_texts)):
    for t , char in enumerate(input_text):
        encoder_input_data[i , t , input_token_dic[char]] = 1
    encoder_input_data[i , t+1: , input_token_dic[' ']] = 1 # Space at last
    
    for t , char in enumerate(target_text):
        decoder_input_data[i , t , target_token_dic[char]] = 1
        # Decoder_target_data will be 1 timestamp behind/offset  
        if t > 0:
            decoder_target_data[i , t-1 , target_token_dic[char]] = 1
    decoder_input_data[i , t+1: , target_token_dic[' ']] = 1
    decoder_target_data[i , t: , target_token_dic[' ']] = 1

In [32]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(20000, 17, 73)
(20000, 59, 99)
(20000, 59, 99)


                                        Model building using LSTM hidden layers

In [33]:
# Defining and processing input sequences
# The return_state contructor argument, configuring a RNN layer to return a list where the first entry
# is the outputs and the next entries are the internal RNN states. This is used to recover the states of the encoder.

encoder_inputs = Input(shape = (None , num_encoder_tokens))
encoder = LSTM(latent_dim , return_state = True)

# We will discard encoder outputs and only takes the states
encoder_outputs , state_h , state_c = encoder(encoder_inputs)
encoder_states = [state_h , state_c]

In [34]:
# Setting up decoder using above states as initial states
# The return_sequences constructor argument, configuring a RNN to return its full sequence of outputs 
#instead of just the last output, which the defaults behavior). This is used in the decoder.
 
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [35]:
# Defining the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [36]:
# Compiling and running the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy' , metrics = ['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          verbose = 2)

Epoch 1/100
250/250 - 135s - loss: 1.0553 - accuracy: 0.7361 - val_loss: 0.8977 - val_accuracy: 0.7490
Epoch 2/100
250/250 - 102s - loss: 0.6871 - accuracy: 0.8033 - val_loss: 0.7400 - val_accuracy: 0.7846
Epoch 3/100
250/250 - 95s - loss: 0.5726 - accuracy: 0.8319 - val_loss: 0.6673 - val_accuracy: 0.8041
Epoch 4/100
250/250 - 97s - loss: 0.5130 - accuracy: 0.8481 - val_loss: 0.6114 - val_accuracy: 0.8195
Epoch 5/100
250/250 - 100s - loss: 0.4718 - accuracy: 0.8594 - val_loss: 0.5748 - val_accuracy: 0.8290
Epoch 6/100
250/250 - 97s - loss: 0.4403 - accuracy: 0.8682 - val_loss: 0.5483 - val_accuracy: 0.8359
Epoch 7/100
250/250 - 95s - loss: 0.4142 - accuracy: 0.8756 - val_loss: 0.5237 - val_accuracy: 0.8429
Epoch 8/100
250/250 - 91s - loss: 0.3917 - accuracy: 0.8817 - val_loss: 0.5099 - val_accuracy: 0.8485
Epoch 9/100
250/250 - 93s - loss: 0.3718 - accuracy: 0.8878 - val_loss: 0.4934 - val_accuracy: 0.8526
Epoch 10/100
250/250 - 95s - loss: 0.3539 - accuracy: 0.8933 - val_loss: 0.4800

Epoch 81/100
250/250 - 97s - loss: 0.0678 - accuracy: 0.9776 - val_loss: 0.7034 - val_accuracy: 0.8676
Epoch 82/100
250/250 - 96s - loss: 0.0670 - accuracy: 0.9776 - val_loss: 0.7103 - val_accuracy: 0.8672
Epoch 83/100
250/250 - 95s - loss: 0.0660 - accuracy: 0.9779 - val_loss: 0.7124 - val_accuracy: 0.8674
Epoch 84/100
250/250 - 97s - loss: 0.0651 - accuracy: 0.9783 - val_loss: 0.7179 - val_accuracy: 0.8674
Epoch 85/100
250/250 - 96s - loss: 0.0642 - accuracy: 0.9784 - val_loss: 0.7189 - val_accuracy: 0.8676
Epoch 86/100
250/250 - 90s - loss: 0.0635 - accuracy: 0.9787 - val_loss: 0.7217 - val_accuracy: 0.8678
Epoch 87/100
250/250 - 89s - loss: 0.0627 - accuracy: 0.9790 - val_loss: 0.7309 - val_accuracy: 0.8666
Epoch 88/100
250/250 - 89s - loss: 0.0620 - accuracy: 0.9792 - val_loss: 0.7333 - val_accuracy: 0.8668
Epoch 89/100
250/250 - 89s - loss: 0.0613 - accuracy: 0.9794 - val_loss: 0.7363 - val_accuracy: 0.8672
Epoch 90/100
250/250 - 104s - loss: 0.0603 - accuracy: 0.9797 - val_loss:

In [39]:
## Encode input and retrieve initiall decoder state
## Run one step of decoder with this inital state
## and a " start of seq " token as target
## Output will be next target token 
## Repeat with current target token and current states

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse lookup token index to decode sequences back 

reverse_input_char_index = dict([(i , char) for char , i in input_token_dic.items()])
reverse_target_char_index = dict([(i , char) for char , i in target_token_dic.items()])



In [41]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_dic['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        print(sampled_token_index)
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_sequence_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

for seq_index in range(50):
    # Taking one sequence at a time for prediction
    input_seq = encoded_input_data[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print("Input sentence : " , input_texts[seq_index])
    print("Decoded sentence : " , decoded_sentence)

39
68
69
56
2
68
61
2
63
52
56
61
67
2
3
1
Input sentence :  Go.
Decoded sentence :  Ouvi un peint !

39
68
69
56
2
68
61
2
63
52
56
61
67
2
3
1
Input sentence :  Go.
Decoded sentence :  Ouvi un peint !

39
68
69
56
2
68
61
2
63
52
56
61
67
2
3
1
Input sentence :  Go.
Decoded sentence :  Ouvi un peint !

39
68
69
56
2
68
61
2
63
52
56
61
67
2
3
1
Input sentence :  Hi.
Decoded sentence :  Ouvi un peint !

39
68
69
56
2
68
61
2
63
52
56
61
67
2
3
1
Input sentence :  Hi.
Decoded sentence :  Ouvi un peint !

39
68
69
56
2
68
61
2
63
52
56
61
67
2
3
1
Input sentence :  Run!
Decoded sentence :  Ouvi un peint !

39
68
69
56
2
68
61
2
63
52
56
61
67
2
3
1
Input sentence :  Run!
Decoded sentence :  Ouvi un peint !

39
68
69
56
2
68
61
2
63
52
56
61
67
2
3
1
Input sentence :  Run!
Decoded sentence :  Ouvi un peint !

39
68
69
56
2
68
61
2
63
52
56
61
67
2
3
1
Input sentence :  Run!
Decoded sentence :  Ouvi un peint !

39
68
69
56
2
68
61
2
63
52
56
61
67
2
3
1
Input sentence :  Run!
Decoded sent